### Cào urls của tất cả các jobs

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Cấu hình Selenium và đường dẫn ChromeDriver
CHROMEDRIVER_PATH = "C:\\Users\\quocn\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"  # Thay bằng đường dẫn ChromeDriver (Windows)


# Hàm lấy liên kết công việc từ một trang
def fetch_job_links_from_page_selenium(base_url, page):
    url = f"{base_url}tat-ca-viec-lam-trang-{page}-vi.html"
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Chạy không giao diện
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Sử dụng Service với ChromeDriver có sẵn
    service = Service(CHROMEDRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    time.sleep(5)  # Đợi cho trang tải xong

    job_links_list = []
    job_links = driver.find_elements(By.CSS_SELECTOR, 'h2 > .job_link')  # Sửa lại CSS selector cho đúng
    job_links_list = [job.get_attribute('href') for job in job_links]

    driver.quit()
    print(f"Lấy được {len(job_links_list)} liên kết công việc từ trang {page}")
    return job_links_list

# Sử dụng Selenium để cào tất cả các trang với đa luồng
def get_all_job_links_selenium_multithread(base_url, start_page=1, max_pages=10, max_workers=5):
    job_links = []
    pages = range(start_page, max_pages + 1)

    # Sử dụng ThreadPoolExecutor để cào đa luồng
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_page = {executor.submit(fetch_job_links_from_page_selenium, base_url, page): page for page in pages}
        for future in as_completed(future_to_page):
            page = future_to_page[future]
            try:
                result = future.result()
                if result:
                    job_links.extend(result)
            except Exception as e:
                print(f"Lỗi khi lấy liên kết từ trang {page}: {e}")

    return job_links

# Sử dụng ví dụ
if __name__ == "__main__":
    BASE_URL = "https://careerviet.vn/viec-lam/"
    job_links = get_all_job_links_selenium_multithread(BASE_URL, max_pages=505, max_workers=10)

    # Lưu liên kết vào tệp
    with open('job_links.txt', 'w', encoding='utf-8') as f:
        for link in job_links:
            f.write(link + '\n')


Lấy được 50 liên kết công việc từ trang 7
Lấy được 50 liên kết công việc từ trang 3
Lấy được 0 liên kết công việc từ trang 8
Lấy được 50 liên kết công việc từ trang 4
Lấy được 50 liên kết công việc từ trang 1
Lấy được 50 liên kết công việc từ trang 6
Lấy được 50 liên kết công việc từ trang 10
Lấy được 50 liên kết công việc từ trang 2
Lấy được 50 liên kết công việc từ trang 5
Lấy được 50 liên kết công việc từ trang 12
Lấy được 50 liên kết công việc từ trang 11
Lấy được 50 liên kết công việc từ trang 13
Lấy được 50 liên kết công việc từ trang 9
Lấy được 50 liên kết công việc từ trang 19
Lấy được 50 liên kết công việc từ trang 16
Lấy được 50 liên kết công việc từ trang 14
Lấy được 50 liên kết công việc từ trang 15
Lấy được 50 liên kết công việc từ trang 17
Lấy được 50 liên kết công việc từ trang 18
Lấy được 50 liên kết công việc từ trang 22
Lấy được 50 liên kết công việc từ trang 21
Lấy được 50 liên kết công việc từ trang 25
Lấy được 50 liên kết công việc từ trang 20
Lấy được 50 liên kết 

### Cào jobs từ URL trong file job_links

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

# Hàm để cào thông tin từ URL công ty (nếu có)
def fetch_company_details(company_url, headers):
    try:
        response = requests.get(company_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lấy tên công ty
        company_name = soup.select_one("h1.name").text.strip() if soup.select_one("h1.name") else "Not specified"

        # Lấy địa điểm công ty
        company_address = (
            soup.select_one("strong:contains('Địa điểm') + p").text.strip()
            if soup.select_one("strong:contains('Địa điểm') + p")
            else "Not specified"
        )

        # Lấy quy mô công ty (loại bỏ phần đầu "Quy mô công ty:")
        company_size_element = soup.select_one("li:has(.mdi-account-supervisor)")
        company_size = (
            company_size_element.text.replace("Qui mô công ty:", "").strip()
            if company_size_element else "Not specified"
        )

        # Lấy loại hình hoạt động (loại bỏ phần đầu "Loại hình hoạt động:")
        company_type_element = soup.select_one("li:has(.mdi-gavel)")
        company_type = (
            company_type_element.text.replace("Loại hình hoạt động:", "").strip()
            if company_type_element else "Not specified"
        )

        # Lấy đường dẫn website công ty
        company_website_element = soup.select_one("li:has(.mdi-link)")
        company_website = company_website_element.text.replace("Website: ", "").strip() if company_website_element else "Not specified"

        # Lấy số lượng người theo dõi
        follow_element = soup.select_one("#countFollowers")
        follower = follow_element.text if follow_element else "Not Found"

        # Lấy giới thiệu công ty
        company_intro_element = soup.select_one(".intro-section .main-text")
        company_intro = company_intro_element.text.strip() if company_intro_element else "Not specified"

        return {
            "Company Name": company_name,
            "Company Address": company_address,
            "Company Scale": company_size,
            "Company Type": company_type,
            "Company Website": company_website,
            "Follower": follower,
            "Company Introduction": company_intro
        }
    except requests.RequestException as e:
        print(f"Error fetching company details from '{company_url}': {e}")
        return {
            "Company Name": "Not specified",
            "Company Address": "Not specified",
            "Company Scale": "Not specified",
            "Company Type": "Not specified",
            "Company Website": "Not specified",
            "Follower": "Not specified",
            "Company Introduction": "Not specified"
        }

# Hàm cào dữ liệu từ một URL công việc
def crawl_single_job(job_url, headers):
    try:
        print(f"Crawling: {job_url}")
        response = requests.get(job_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        time.sleep(random.uniform(1, 3))  # Chờ ngẫu nhiên để tránh bị chặn

        # Lấy tên công việc
        job_name = soup.select_one('h1.title').text.strip() if soup.select_one('h1.title') else "Not specified"

        # Lấy thông tin lương
        salary_info = (
            soup.select_one("strong:contains('Lương') + p").text.strip()
            if soup.select_one("strong:contains('Lương') + p")
            else "Not specified"
        )

        # Lấy địa điểm
        location_elements = soup.select('.map p a')
        locations = ', '.join([loc.text.strip() for loc in location_elements]) if location_elements else "Not specified"

        # Lấy các thông tin khác
        experience = (
            soup.select_one("strong:contains('Kinh nghiệm') + p").text.strip()
            if soup.select_one("strong:contains('Kinh nghiệm') + p")
            else "Not specified"
        )
        job_level = (
            soup.select_one("strong:contains('Cấp bậc') + p").text.strip()
            if soup.select_one("strong:contains('Cấp bậc') + p")
            else "Not specified"
        )
        deadline = (
            soup.select_one("strong:contains('Hết hạn nộp') + p").text.strip()
            if soup.select_one("strong:contains('Hết hạn nộp') + p")
            else "Not specified"
        )
        update_date = (
            soup.select_one("strong:contains('Ngày cập nhật') + p").text.strip()
            if soup.select_one("strong:contains('Ngày cập nhật') + p")
            else "Not specified"
        )
        employment_type = (
            soup.select_one("strong:contains('Hình thức') + p").text.strip()
            if soup.select_one("strong:contains('Hình thức') + p")
            else "Not specified"
        )

        # Lấy loại ngành nghề
        briefcase_element = soup.select_one("li:has(.mdi-briefcase)")
        list_briefcase = ", ".join([a.text for a in briefcase_element.find_all("a")]) if briefcase_element else "Not Found"

        # Lấy danh sách phúc lợi
        welfare_elements = soup.select("ul.welfare-list li")
        welfare_list = ', '.join([welfare.text.strip() for welfare in welfare_elements]) if welfare_elements else "Not specified"

        # Lấy phần "Mô tả Công việc"
        job_description_section = soup.select("h2:contains('Mô tả Công việc') ~ *")
        job_description_list = []
        for section in job_description_section:
            if section.name == "p" and section.text.strip():
                job_description_list.append(section.text.strip())
            elif section.name == "ul":
                description_items = section.find_all("li")
                job_description_list.extend([item.text.strip() for item in description_items if item.text.strip()])
        job_description = '\n'.join(job_description_list) if job_description_list else "Not specified"

        # Lấy phần "Yêu Cầu Công Việc"
        job_requirements_section = soup.select("h2:contains('Yêu Cầu Công Việc') ~ *")
        job_requirements_list = []
        for section in job_requirements_section:
            if section.name == "p" and section.text.strip():
                job_requirements_list.append(section.text.strip())
            elif section.name == "ul":
                requirements_items = section.find_all("li")
                job_requirements_list.extend([item.text.strip() for item in requirements_items if item.text.strip()])
        job_requirements = '\n'.join(job_requirements_list) if job_requirements_list else "Not specified"

        # Lấy nội dung của phần "Thông tin khác"
        other_info_section = soup.select_one("h3:contains('Thông tin khác') + div ul")
        other_info = '\n'.join([item.text.strip() for item in other_info_section.find_all("li")]) if other_info_section else "Not specified"

        # Lấy danh sách các Job Tags / Skills
        job_tags_elements = soup.select("div.job-tags ul li")
        job_tags = ', '.join([tag.text.strip() for tag in job_tags_elements]) if job_tags_elements else "Not specified"

        # Lấy URL công ty từ điều hướng
        company_url_element = soup.select_one("#tabs-job-company a")
        company_url = company_url_element['href'] if company_url_element else "Not Found"
        company_details = fetch_company_details(company_url, headers) if company_url != "Not Found" else {}

        # Tạo thông tin công việc
        job_information = {
            "Job Name": job_name,
            "Job Link": job_url,
            "Location": locations,
            "Update Date": update_date,
            "Deadline": deadline,
            "Experience": experience,
            "Job Level": job_level,
            "Industry": list_briefcase,
            "Employment Type": employment_type,
            "Welfare": welfare_list,
            "Job Description": job_description,
            "Job Requirements": job_requirements,
            "Other Information": other_info,
            "Job Tags": job_tags,
            "Company URL": company_url,
            **company_details,  # Thêm chi tiết công ty
            "Salary": salary_info
        }
        return job_information

    except requests.RequestException as e:
        print(f"Error fetching data from '{job_url}': {e}")
        return None

def crawl_job_information(job_urls):
    job_information_list = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    total_urls = len(job_urls)
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = [executor.submit(crawl_single_job, url, headers) for url in job_urls]
        for i, future in enumerate(as_completed(futures), 1):
            result = future.result()
            if result:
                job_information_list.append(result)
            # Print progress
            print(f"Crawled {i}/{total_urls} URLs")
    
    return pd.DataFrame(job_information_list)


# Đọc các link từ file txt
with open('job_links.txt', 'r', encoding='utf-8') as f:
    job_links_from_file = [line.strip() for line in f]

# Crawl job information based on collected job links
job_df = crawl_job_information(job_links_from_file)

# Print the DataFrame
job_df

Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-kho-van.35C2670B.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-kinh-doanh-phan-mem-quan-tri-doanh-nghiep.35C27217.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh-san-pham-phan-mem-erp.35C27216.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/data-analyst.35C27215.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/data-engineer.35C27214.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/business-analyst.35C27213.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/automation-tester.35C27212.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/software-project-manager-pm.35C27211.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/qc-manual-tester.35C27210.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-kiem-thu-phan-mem-tester.35C2720F.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/key-account-executive-chuoi-nha-thuoc.35C2720D.html
Crawling: https://car

c:\Users\quocn\anaconda3\Lib\site-packages\soupsieve\css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-cham-soc-ho-hai-san.35C1C0D4.htmlCrawling: https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-quan-ly-lop-hoc-truc-tuyen.35C1BE60.html
Crawled 1/25000 URLs
Crawled 2/25000 URLs

Crawling: https://careerviet.vn/vi/tim-viec-lam/truong-phong-hanh-chinh-nhan-su.35C1BE80.html
Crawled 3/25000 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/sale-team-leader.35C1BE74.html
Crawled 4/25000 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/automation-project-engineer.35C1BF12.html
Crawled 5/25000 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/cua-hang-truong-aeon-mall-tanphu.35C1BE5D.html
Crawled 6/25000 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/hieu-pho-mam-non-khoi-truong-mam-non-cu-chi-ho-chi-minh.35C1BF62.html
Crawled 7/25000 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-hanh-chinh-nhan-su-lam-audit-tieng-anh.35C1BED0.html
Crawled 8/25000 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/tele

In [2]:
job_df.to_csv('job_data.csv', index=False, encoding='utf-8-sig')